## Gather Data

In [1]:
import requests
import csv
import pandas as pd
import shlex

In [215]:
response = requests.get(
    url = "https://api.data.gov.in/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69?api-key=579b464db66ec23bdd0000013e72a63a8d8d46297563eb47977e01cd&format=csv&offset=0&limit=1000"
)

In [216]:
data = response.content.decode('utf-8').split('\n')

In [217]:
data = [shlex.split(a.replace(',', ' ')) for a in data]

In [218]:
df = pd.DataFrame(data[1:], columns=data[0])

In [219]:
df.drop(index=822, inplace=True)

In [221]:
df.replace({
    'Andhra_Pradesh': 'Andhra Pradesh',
    'Uttar_Pradesh': 'Uttar Pradesh',
    'West_Bengal': 'West Bengal'    
}, inplace=True)

Now I need latitude longitude of these places

In [223]:
import geopy

In [248]:
from geopy.extra.rate_limiter import RateLimiter
temp = ""
def lat_long(city, state, index):
    global temp
    global location
    loc = city + ", " + state
    if loc == temp:
        try:
            return str(location.latitude) + "," + str(location.longitude)
        except:
            return None
    else:
        print(loc)
        location = geolocator.geocode(loc)
        temp = loc
        try:
            return str(location.latitude) + "," + str(location.longitude)
        except:
            return None

geocode = RateLimiter(lat_long, min_delay_seconds=0.5)
temp = ""

In [ ]:
df['lat_long'] = df.apply(lambda x: geocode(x['city'], x['state'], x.index), axis=1)

In [250]:
df.head()

,id,country,state,city,station,last_update,pollutant_id,pollutant_min,pollutant_max,pollutant_avg,pollutant_unit,lat_long
0,1,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,PM2.5,128,312,219,NA,"16.5744201,80.3556079"
1,2,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,PM10,118,214,146,NA,"16.5744201,80.3556079"
2,3,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,NO2,47,157,76,NA,"16.5744201,80.3556079"
3,4,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,NH3,3,5,5,NA,"16.5744201,80.3556079"
4,5,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,SO2,16,35,27,NA,"16.5744201,80.3556079"


In [ ]:
df.to_csv('data.csv')

## Plotting

In [129]:
df = pd.read_csv('data.csv', dtype={'lat_long': 'str'})

In [130]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [131]:
df.head()

,id,country,state,city,station,last_update,pollutant_id,pollutant_min,pollutant_max,pollutant_avg,pollutant_unit,lat_long
0,1,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,PM2.5,128.0,312.0,219.0,NaN,"16.5744201,80.3556079"
1,2,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,PM10,118.0,214.0,146.0,NaN,"16.5744201,80.3556079"
2,3,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,NO2,47.0,157.0,76.0,NaN,"16.5744201,80.3556079"
3,4,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,NH3,3.0,5.0,5.0,NaN,"16.5744201,80.3556079"
4,5,India,Andhra Pradesh,Amaravati,Secretariat Amaravati - APPCB,15-12-2018 02:00:00,SO2,16.0,35.0,27.0,NaN,"16.5744201,80.3556079"


In [132]:
plotdf = df.iloc[:, [2,3,6,9,11]]

In [133]:
plot_df = plotdf.sort_values(by='pollutant_id')

In [134]:
plot_df.dropna(axis = 0, inplace=True)

In [135]:
lat = [float(x.split(',')[0]) for x in plot_CO['lat_long']]
long = [float(x.split(',')[1]) for x in plot_CO['lat_long']]

In [136]:
import plotly
plotly.tools.set_credentials_file(username='edmdopes', api_key='JlrgiM5KUuQYXyMPCSHC')

In [137]:
import plotly.plotly as py

In [138]:
data = []
for a in plot_df['pollutant_id'].unique():
    plot = plot_df[plot_df['pollutant_id'] == f'{a}']
    lat = [float(x.split(',')[0]) for x in plot['lat_long']]
    long = [float(x.split(',')[1]) for x in plot['lat_long']]
    text = plot['city'] + ", " + plot['state'] + '-' + a + '-' + plot['pollutant_avg'].astype('str')
    p_dict = dict(
        type = 'scattergeo',
        locationmode = 'India',
        lon = long,
        lat = lat,
        text = text,
        mode = 'markers',
        marker = dict(
            size = 20,
            opacity = 0.8,
            reversescale = False,
            autocolorscale = True,
            symbol = 'circle',
            cmin = 0,
            color = plot['pollutant_avg'],
            cmax = plot['pollutant_avg'].max(),
            colorbar=dict(
                size = 20,
                title=f"{a} as pollutant"
            )
        ))
    data.append(p_dict)

In [139]:
layout = dict(
        title = 'Pollution in India',
        colorbar = True,
        geo = dict(
            scope='asia',
            projection=dict( type='delhi india' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.8,
            subunitwidth = 0.8,
            center = dict(
                lat = 23.61097367916291,
                lon = 80.3555029509925
            ),
            margin = dict(
                t = 55,
                b = 55,
                l = 55,
                r = 55,
                p = 0,
                autoexpand = True
            )
        ),
    )

In [140]:
fig = dict( data=[data[0]], layout=layout )
py.iplot( fig, validate=False, filename='CO' )

In [141]:
fig = dict( data=[data[1]], layout=layout )
py.iplot( fig, validate=False, filename='NH3' )

In [142]:
fig = dict( data=[data[2]], layout=layout )
py.iplot( fig, validate=False, filename='NO2' )

In [143]:
fig = dict( data=[data[3]], layout=layout )
py.iplot( fig, validate=False, filename='OZONE' )

In [144]:
fig = dict( data=[data[4]], layout=layout )
py.iplot( fig, validate=False, filename='PM10' )

In [145]:
fig = dict( data=[data[5]], layout=layout )
py.iplot( fig, validate=False, filename='PM2.5' )

In [146]:
fig = dict( data=[data[6]], layout=layout )
py.iplot( fig, validate=False, filename='SO2' )